# 평가자료 22년 4월 20일

- 데이터 가져오기

In [157]:
import requests, re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import quote
import folium, json

In [158]:
url = 'https://minwon.police.go.kr/#customerCenter/plcstSearch'
header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"}
req = requests.get(url, headers = header) 
driver = webdriver.Chrome('chromedriver.exe')
html = req.text
driver.get(url)
time.sleep(2)

In [159]:
driver.find_element_by_xpath('//*[@id="tab3"]').click()
time.sleep(2)

In [160]:
driver.find_element_by_xpath('//*[@id="indexedA"]').click()   
time.sleep(2)

In [161]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
list = soup.select_one('.table-list.delStr')
list_detail = list.find('tbody')
trs = soup.select('#minwonTbody > tr')
trs

[<tr> <td>경찰청 서울특별시경찰청 서울중부경찰서 </td>
 <td class="title">서울특별시 중구 수표로 27</td>
 <td>
 <a addr="서울특별시 중구 수표로 27" href="#" id="viewMap" latd="37.563630778711754" lotd="126.99002154542026" plcstid="3" plcstnm="경찰청 서울특별시경찰청 서울중부경찰서">
 <img alt="지도보기" src="/images/icon/icon_map.gif"/>
 </a>
 </td></tr>,
 <tr> <td>경찰청 서울특별시경찰청 서울종로경찰서 </td>
 <td class="title">서울특별시 종로구 율곡로 46</td>
 <td>
 <a addr="서울특별시 종로구 율곡로 46" href="#" id="viewMap" latd="37.57587651959812" lotd="126.98454988134327" plcstid="4" plcstnm="경찰청 서울특별시경찰청 서울종로경찰서">
 <img alt="지도보기" src="/images/icon/icon_map.gif"/>
 </a>
 </td></tr>,
 <tr> <td>경찰청 서울특별시경찰청 서울남대문경찰서 </td>
 <td class="title">서울특별시 중구 한강대로 410</td>
 <td>
 <a addr="서울특별시 중구 한강대로 410" href="#" id="viewMap" latd="37.55481288235392" lotd="126.97354934704215" plcstid="5" plcstnm="경찰청 서울특별시경찰청 서울남대문경찰서">
 <img alt="지도보기" src="/images/icon/icon_map.gif"/>
 </a>
 </td></tr>,
 <tr> <td>경찰청 서울특별시경찰청 서울서대문경찰서 </td>
 <td class="title">서울특별시 서대문구 통일로 113</td>
 <td>
 <a addr="서울특

In [162]:
tr = trs[0]
name_long = tr.find('td').get_text().strip()
name = name_long.split()[-1]
name

'서울중부경찰서'

In [163]:
addr = tr.find('td', {'class':'title'}).get_text()
addr

'서울특별시 중구 수표로 27'

In [164]:
lines = []
for tr in trs:
    name_long = tr.find('td').get_text().strip()
    name = name_long.split()[-1]
    addr = tr.find('td', {'class':'title'}).get_text()
    lines.append([name,addr])
lines  

[['서울중부경찰서', '서울특별시 중구 수표로 27'],
 ['서울종로경찰서', '서울특별시 종로구 율곡로 46'],
 ['서울남대문경찰서', '서울특별시 중구 한강대로 410'],
 ['서울서대문경찰서', '서울특별시 서대문구 통일로 113'],
 ['서울혜화경찰서', '서울특별시 종로구 창경궁로 112-16']]

In [165]:
from tqdm.notebook import tqdm

In [166]:
lines = []

for i in tqdm(range(0,6)):
        for tr in trs:
            name_long = tr.find('td').get_text().strip()
            name = name_long.split()[-1]
            addr = tr.find('td', {'class':'title'}).get_text()
            lines.append([name,addr])
        driver.find_element_by_css_selector('.next').click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        trs = soup.select('#minwonTbody > tr')
        time.sleep(1)

  0%|          | 0/6 [00:00<?, ?it/s]

In [167]:
driver.close()

In [168]:
df = pd.DataFrame(lines, columns=['경찰서명','주소'])
df

,경찰서명,주소
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16
5,서울중부경찰서,서울특별시 중구 수표로 27
6,서울종로경찰서,서울특별시 종로구 율곡로 46
7,서울남대문경찰서,서울특별시 중구 한강대로 410
8,서울서대문경찰서,서울특별시 서대문구 통일로 113
9,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


In [169]:
df[df.경찰서명.isin(["서울서부경찰서","서울강동경찰서"])].index

Int64Index([18, 25], dtype='int64')

In [170]:
df.주소[18] = '서울특별시 은평구 서오릉로 58'
df.주소[25] = '서울 강동구 양재대로96길 35'

In [171]:
df.to_csv('서울경찰서2.csv', index=False)

In [172]:
df = pd.read_csv('서울경찰서2.csv')
df.head()

,경찰서명,주소
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


In [173]:
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

In [174]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [175]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))

In [181]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head(5)

,경찰서명,주소,위도,경도
0,서울중부경찰서,서울특별시 중구 수표로 27,37.563632,126.990021
1,서울종로경찰서,서울특별시 종로구 율곡로 46,37.575876,126.984551
2,서울남대문경찰서,서울특별시 중구 한강대로 410,37.554811,126.973549
3,서울서대문경찰서,서울특별시 서대문구 통일로 113,37.565085,126.967074
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16,37.571768,126.999273


In [182]:
df.to_csv('서울경찰서2.csv', index=False)

In [183]:
filename = '../04.지도시각화/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [184]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        icon = folium.Icon(color='black', icon='flag'),
        popup=folium.Popup(df.주소[i], max_width=350),
        tooltip=df.경찰서명[i]
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울소재 경찰서 위치</h3>'    
map.get_root().html.add_child(folium.Element(title_html)) 
map